In [1]:
import os
import torch

from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
    logging,
    BitsAndBytesConfig
)
from trl import SFTTrainer
from peft import LoraConfig

# Set all 4 GPUs visible
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:

# Load tokenizer and model
model_name = "Qwen/Qwen2.5-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)
model.gradient_checkpointing_enable()
model.config.use_cache = False

# Load and split dataset
data = load_dataset(
    "json",
    data_files={
        "train": "updated_simplified_recipes_with_adaptations.json",
        "validation": "updated_simplified_recipes_with_adaptations.json"
    }
)
dataset_train = data['train']
dataset_valid = data['validation']

print(dataset_train)
print(dataset_valid)

# Preprocessing function
def preprocess_function(example):
    """
    Formatting function returning a list of samples (kind of necessary for SFT API).
    """
    text = f"### Find products that can be replaced by other:\n{example['steps']}\n\n### Input:\n{example['ingredients']}\n\n### Response:\n{example['adaptation_tips']}"
    return text

training_args = TrainingArguments(
    output_dir=f"results/logs",
    evaluation_strategy='epoch',
    weight_decay=0.01,
    load_best_model_at_end=True,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    logging_strategy='epoch',
    save_strategy='epoch',
    num_train_epochs=7,
    save_total_limit=2,
    bf16=True,
    fp16=False,
    report_to='tensorboard',
    dataloader_num_workers=1,
    gradient_accumulation_steps=8,
    learning_rate=5e-5,
    lr_scheduler_type='constant',
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_train,
    eval_dataset=dataset_valid,
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_args,
    formatting_func=preprocess_function,
    packing=True
)
# Clear GPU memory
history = trainer.train()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset({
    features: ['ingredients', 'steps', 'adaptation_tips'],
    num_rows: 100
})
Dataset({
    features: ['ingredients', 'steps', 'adaptation_tips'],
    num_rows: 100
})


/home/gituser/.local/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/gituser/.local/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, packing. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/gituser/.local/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/gituser/.local/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:212: UserWarning: You passed a `packing` argument to the SFTTrainer, the val

[2024-11-25 08:18:20,961] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


Epoch,Training Loss,Validation Loss
0,3.654600,1.511212
1,2.784000,1.312192
2,2.437700,1.188061
3,0.876900,0.804454
4,1.424400,0.663396
5,0.403600,0.563008


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


In [2]:
recipe = '''Here’s a recipe using meat, pasta, milk, and bread: Creamy Pasta with Meat Sauce and Crispy Breadcrumbs.  

### Ingredients:  
- Pasta: 300 g (spaghetti, fettuccine, or penne).  
- Meat: 300 g (ground beef, pork, or chicken).  
- Milk: 200 ml.  
- Bread: 2 slices (stale bread works best).  
- Onion: 1 (finely chopped).  
- Garlic: 2 cloves (minced).  
- Cheese: 50 g (such as Parmesan or another hard cheese).  
- Butter: 2 tbsp.  
- Vegetable oil: 2 tbsp.  
- Flour: 1 tbsp.  
- Spices: Salt, pepper, oregano, paprika (to taste).  

### Instructions:  

#### 1. Prepare the Breadcrumbs:  
1. Cut the bread into cubes and crumble it into breadcrumbs (using your hands or a blender).  
2. Heat 1 tbsp of butter in a pan.  
3. Toast the breadcrumbs until golden and crispy, seasoning with a pinch of salt and your favorite spices. Set aside.  

#### 2. Make the Meat Sauce:  
1. Heat the vegetable oil in a pan.  
2. Sauté the chopped onion and garlic until soft.  
3. Add the ground meat and cook until browned, breaking it into small pieces.  
4. Season with salt, pepper, oregano, and paprika.  

#### 3. Prepare the Creamy Sauce:  
1. In a separate pan, melt 1 tbsp of butter.  
2. Add the flour and cook for 1–2 minutes, stirring constantly.  
3. Gradually pour in the milk while stirring to avoid lumps.  
4. Cook until thickened, then add the grated cheese and mix well.  

#### 4. Combine Everything:  
1. Cook the pasta in salted water until *al dente*.  
2. Toss the pasta with the meat sauce, then stir in the creamy sauce.  

#### 5. Serve:  
1. Plate the pasta and sprinkle generously with crispy breadcrumbs.  
2. Add a bit more grated cheese on top if desired.  

Enjoy your meal!"
'''

In [3]:
prompt = f"Product that we have: pork 300g, milk 500 ml, pasta 200g, bread 100g, mayonnaise 50g. Base recipe: {recipe} Use only these products and give me recipe"
messages = [
    {"role": "system", "content": "You are Qwen - a helpful assistant for generating recipes based on source products."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=1024
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


In [4]:
print(response)

Certainly! Given the ingredients you have—pork 300g, milk 500ml, pasta 200g, bread 100g, and mayonnaise 50g—we can adjust the original recipe to fit these exact items. We'll need to make some adjustments since we don't have onions, garlic, cheese, butter, or vegetable oil. However, we can still create a delicious dish by using what we have.

### Adjusted Recipe: Creamy Pork Pasta with Crispy Breadcrumbs

#### Ingredients:
- Pasta: 200g (spaghetti, fettuccine, or penne)
- Pork: 300g (ground pork)
- Milk: 500ml
- Bread: 100g (preferably stale bread)
- Mayonnaise: 50g

#### Instructions:

##### 1. Prepare the Breadcrumbs:
1. Cut the bread into cubes and crumble it into breadcrumbs.
2. Heat a non-stick pan over medium heat without any oil or butter.
3. Toast the breadcrumbs until golden and crispy, seasoning with a pinch of salt and pepper. Set aside.

##### 2. Make the Pork Sauce:
1. Heat a non-stick pan over medium heat.
2. Add the ground pork and cook until browned, breaking it into sma